<a href="https://colab.research.google.com/github/cheatham1/NLP/blob/main/Finetuning_and_Evaluating_a_BERT_model_for_classification_using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EU-JAV - Finetuning and Evaluating a BERT model for Classification



In this notebook we will finetune BERT base models.


After training, we will save the model, evaluate it and use it for predictions.


Thanks to Per from the National Library of Norway

In [ ]:
# Runtime > Change runtime type menu - GPU

#gpu_info = !nvidia-smi
#if gpu_info.find('failed') >= 0:
#gpu_info = '\n'.join(gpu_info)
#  print('Not connected to a GPU')
#else:
#  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!pip install --upgrade transformers
!pip install sentencepiece
!pip install accelerate -U

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import EarlyStoppingCallback
#from transformers import BertForPreTraining
#from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
#from transformers import CamembertModel, CamembertTokenizer
#from transformers import T5Tokenizer, T5ForConditionalGeneration

from transformers import RobertaTokenizer, XLMRobertaXLForSequenceClassification

#from transformers import ElectraForSequenceClassification
#from transformers import XLMTokenizer, XLMWithLMHeadModel
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification


In [ ]:
#!pip install transformers

#print(transformers.__version__)

In [ ]:
#@markdown Set the main model that the training should start from
#
#model_name = 'bert-base-multilingual-uncased' #@param ["NbAiLab/nb-bert-base", "NbAiLab/nb-bert-large", "bert-base-multilingual-cased", "bert-base-multilingual-uncased"]
#model_name = 'm-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0' #@param ["dbmdz/bert-base-italian-xxl-uncased", "dbmdz/bert-base-italian-xxl-cased","m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0","bert-base-multilingual-cased","bert-base-multilingual-uncased"]
#model_name = 'digitalepidemiologylab/covid-twitter-bert-v2'
#model_name = 'dbmdz/bert-base-italian-xxl-uncased' #@param ["dbmdz/bert-base-italian-xxl-uncased","m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0","bert-base-uncased", "bert-base-multilingual-cased"]
#model_name = 'xlm-roberta-base' #@param['roberta-base','xlm-roberta-base','roberta-large-mnli','cardiffnlp/twitter-roberta-base', 'dbmdz/bert-base-italian-uncased']


#model_name = "google/electra-small-discriminator"
#model_name = "dbmdz/electra-base-italian-xxl-cased-discriminator"
model_name = "xlm-roberta-large"
#model_name = "xlm-roberta-xlarge"
#model_name = "xlm-roberta-base"
#odel_name = "dbmdz/bert-base-italian-xxl-uncased"

#@markdown ---
#@markdown Set training parameters
batch_size =  16 #16@param {type: "integer"}
learning_rate = 2e-5 #@param {type: "number"}
warmup_proportion = 0.15 #@param {type: "number"}

num_epochs = 15 #@param {type: "integer"} # 13
max_seq_length = 96 #256 , 128, 98, 128@param {type: "integer"}
weight_decay = 0.01 #@param {type: "number"} # 0.01

# Electra
#tokenizer = BertTokenizer.from_pretrained(model_name)
#model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels = 3)

# BERT
#tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
#model = BertModel.from_pretrained(model_name, num_labels=3)

# roberta-base
#tokenizer = RobertaTokenizer.from_pretrained(model_name)
#model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

# xlm-roberta-large and base **
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)

#tokenizer = RobertaTokenizer.from_pretrained("xlm-roberta-xlarge")
#model = XLMRobertaXLForSequenceClassification.from_pretrained("xlm-roberta-xlarge", problem_type="multi_label_classification")

# xlm-roberta-large
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForMaskedLM.from_pretrained(model_name, num_labels=3)

#tokenizer = CamembertTokenizer.from_pretrained(model_name)
#model = CamembertModel.from_pretrained(model_name, num_labels=3)

#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModel.from_pretrained(model_name)
#model = AutoModelForMaskedLM.from_pretrained(model_name)


### finetunig data has 3 or 4 labels 0=promotional,1=neutral,2=discouraging,3=contro-no-vax


In [ ]:
removeUserLink = False
addData = True  # True = use dataset1+2
useUnSplitData = False
#useRemovedDifficultTrainingData = False # datset saved after initial run to remove difficult tweets
useSpanishData = False

In [ ]:
vocab = tokenizer.convert_ids_to_tokens(range(tokenizer.vocab_size))
print(len(vocab))

In [ ]:
words = ["maschera", "mask", "covid", "coronavirus", "virus", "isolation", "confinement", "vaccination", "vaccine"]
for word in words:
  print(word, (word in vocab))

In [ ]:
#print(len(tokenizer.vocab))

## Load and Prepare the Dataset used for Finetuning
The selected dataset is loaded directly from a web resource. It is coded with labels and text in a comma-separated file. You can replace this with any other data source. This data is here converted into the pytorch data format.

In [ ]:
if useUnSplitData:

  print("Use full datasets and split")
  # ----- Data -----#
  data1 = pd.read_csv('https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_dataset1_5categories.csv',
    names=["label", "text"]
  )

  data2 = pd.read_csv('https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_dataset2_5categories.csv',
    names=["label", "text"]
  )

  # remove ambiguous
  data1.drop( data1[ data1['label'] == 3].index, inplace=True)
  data2.drop( data2[ data2['label'] == 3].index, inplace=True)

  # remove indiscernable
  data1.drop( data1[ data1['label'] == 4].index, inplace=True)
  data2.drop( data2[ data2['label'] == 4].index, inplace=True)

  #combine indiscernable and neutral
  #data1['label'] = data1['label'].replace(4, 1) # indiscernable with neutral
  #data2['label'] = data2['label'].replace(4, 1) # indiscernable with neutral

  #combine indiscernable and ambiguous
  #data1['label'] = data1['label'].replace(4, 3) # indiscernable with neutral
  #data2['label'] = data2['label'].replace(4, 3) # indiscernable with neutral

  from sklearn.model_selection import train_test_split

  train_data1, devtest_dataset  = train_test_split(data1, train_size=0.6)
  dev_data1, test_data1         = train_test_split(devtest_dataset, train_size=0.5)

  train_data2, devtest_dataset2  = train_test_split(data2, train_size=0.6)
  dev_data2, test_data2       = train_test_split(devtest_dataset2, train_size=0.5)

  print(train_data1.shape, dev_data1.shape, test_data1.shape)
  print(train_data2.shape, dev_data2.shape, test_data2.shape)

In [ ]:
#data1.groupby(['label']).count()

In [ ]:
#data2.groupby(['label']).count()

In [ ]:
# ----- Data set 1 Aleady split into train, dev, test-----#

train_data1 = pd.read_csv(
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/datasetA_train_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/july_train_3categories_2.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/dataset1_train_3categories_noreplies_combinedIndiscNeutral.csv',

    #names=["label", "text"]
    names=['Annotator1','Annotator2','Annotator3','label','text clean','index']
)
dev_data1 = pd.read_csv(
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/datasetA_dev_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/july_dev_3categories_2.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/dataset1_dev_3categories_noreplies_combinedIndiscNeutral.csv',

    #names=["label", "text"]
    names=['Annotator1','Annotator2','Annotator3','label','text clean','index']

)
test_data1 = pd.read_csv(
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/datasetA_test_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/july_test_3categories_2.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/dataset1_test_3categories_noreplies_combinedIndiscNeutral.csv',

    #names=["label", "text"]
    names=['Annotator1','Annotator2','Annotator3','label','text clean','index']
)


print("Dataset1: ", train_data1.shape, dev_data1.shape, test_data1.shape)





In [ ]:
# ----- Data set 2 Aleady split into train, dev, test-----#

train_data2 = pd.read_csv(
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/newconsistent_oct21_train_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/newconsistent_oct_570_train_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/norepeats_oct_train_3categories_2.csv',
    #####'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_oct_train_3categories_2.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/dataset2_train_3categories_noreplies_combinedIndiscNeutral.csv',
    #names=["label", "text"]

    'https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/datasetB_train_3categories.csv',
    names=['Annotator1','Annotator2','Annotator3','label','text clean','index']
    )


dev_data2 = pd.read_csv(
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/newconsistent_oct21_dev_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/newconsistent_oct_570_dev_3categories.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/norepeats_oct_dev_3categories_2.csv',
    #####'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_oct_dev_3categories_2.csv',
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/dataset2_dev_3categories_noreplies_combinedIndiscNeutral.csv',

    #names=["label", "text"]
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/datasetB_dev_3categories.csv',
    names=['Annotator1','Annotator2','Annotator3','label','text clean','index']
)


test_data2 = pd.read_csv(
    #'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/no_replies_no_mentions_oct_test_3categories_2.csv',
    #names=["label", "text"]

    'https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/datasetB_test_3categories.csv',
    names=['Annotator1','Annotator2','Annotator3','label','text clean','index']
)

print("Dataset2: ", train_data2.shape, dev_data2.shape, test_data2.shape)


In [ ]:
# Relabelled dev data

dev_data1_r = pd.read_csv(
    "https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/DevDataA_DisagreeLabelsAndPredictions_relabeled.csv",
    names=["text clean","label","index"], skiprows = 1
)

dev_data2_r = pd.read_csv(
    "https://raw.githubusercontent.com/cheatham1/EU-JAV-AB/main/3categories/DevDataB_DisagreeLabelsAndPredictions_relabeled.csv",
    names=["text clean","label","index"], skiprows = 1
)
print("Dataset relabelled: ", dev_data1_r.shape, dev_data2_r.shape)


In [ ]:
dev_data1_r.groupby(['label']).count()

In [ ]:
dev_data2_r.groupby(['label']).count()

In [ ]:
dev_data1_r.head()

In [ ]:
if addData == True:
  print("adding datasets")

  train_data = train_data1.append(train_data2)
  dev_data  = dev_data1.append(dev_data2)
  test_data  = test_data1.append(test_data2)

else:
  train_data= train_data1.copy()
  dev_data= dev_data1.copy()
  test_data= test_data1.copy()

In [ ]:
# use relabelled dev data
# dev_data  = dev_data1_r.append(dev_data2_r)


In [ ]:
train_data_spanish = pd.read_csv(
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/train_spanish_dataset.csv',
    names=["label", "text"])

dev_data_spanish = pd.read_csv(
    'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/dev_spanish_dataset.csv',
    names=["label", "text"])


In [ ]:
train_data.groupby(['label']).count()

In [ ]:
if useSpanishData:
  train_data = train_data.append(train_data_spanish)
  dev_data = dev_data.append(dev_data_spanish)

In [ ]:
#if useRemovedDifficultTrainingData == True:
#  test_data = pd.read_csv(
#      'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/trainingDataReduced_10epochs_1007.csv',
#      names=["label", "text"])


In [ ]:
#!pip install emoji
#import emoji
#train_data['text'] = train_data['text'].apply(lambda x: emoji.demojize(x))
#dev_data['text']   = dev_data['text'].apply(lambda x: emoji.demojize(x))
#test_data['text']  = test_data['text'].apply(lambda x: emoji.demojize(x))

In [ ]:
train_data.groupby(['label']).count()

In [ ]:
dev_data.groupby(['label']).count()

In [ ]:
print(f'The dataset is imported.\n\nThe training dataset has {len(train_data)} items.\nThe development dataset has {len(dev_data)} items. \nThe test dataset has {len(test_data)} items')


In [ ]:
train_data.label = train_data.label.astype('float').astype('Int64')
dev_data.label = dev_data.label.astype('float').astype('Int64')
test_data.label = test_data.label.astype('float').astype('Int64')

In [ ]:
#X_dev = list(dev_data["text"])
#X_test = list(test_data["text"])

#X_dev_tokenized = tokenizer(X_dev, padding=True, truncation=True, max_length=120)
#X_test_tokenized = tokenizer(X_dev, padding=True, truncation=True, max_length=120)


In [ ]:
#def removeUserLink(df):
#  df['text clean'] = df.text.str.replace('user','')
#  df['text clean'] = df.text.str.replace('link','')
#  return(df)

#if removeUserLink:
#  dev_data = removeUserLink(dev_data)
##  train_data = removeUserLink(train_data)
#  test_data = removeUserLink(test_data)


In [ ]:
#train_data['text'] = train_data.text.str.replace('\n',' ')
#dev_data['text'] = dev_data.text.str.replace('\n',' ')
#test_data['text'] = test_data.text.str.replace('\n',' ')

In [ ]:
train_data = train_data.sample(frac=1).reset_index(drop=True)
dev_data = dev_data.sample(frac=1).reset_index(drop=True)

In [ ]:
## ----- Preprocess data -----#

X_train = list(train_data["text clean"])
y_train = list(train_data["label"])
X_val = list(dev_data["text clean"])
y_val = list(dev_data["label"])
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=max_seq_length)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=max_seq_length)

# Create torch dataset


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

print(f'The dataset is imported.\n\nThe training dataset has {len(train_data)} items.\nThe development dataset has {len(dev_data)} items. \nThe test dataset has {len(test_data)} items')
steps = round(len(train_data)/batch_size)

num_warmup_steps = round(steps*warmup_proportion*num_epochs)
print(f'You are planning to train for a total of {steps} steps * {num_epochs} epochs = {num_epochs*steps} steps. Warmup is {num_warmup_steps} steps or {round(100*num_warmup_steps/(steps*num_epochs))}%. We recommend at least 10%.')


In [ ]:
totalsize = len(train_data) + len(dev_data) + len(test_data)
print(f'The dataset total size {totalsize}')

In [ ]:
#X_train[0]

In [ ]:
#sample = "coronavirus: vaccino usa, il professor bucci sulla sperimentazione di moderna"
#sample = "coronavirus: modern testing”
#sample= "https://t.co/Ub5NJddOYm"
#sample = "https://t.co/YfZ7jhi1mN"

#print(encoding)
##encoding = tokenizer.encode(sample)
#print(tokenizer.convert_ids_to_tokens(encoding))


# Start Training
We are here using the HuggingFace Trainer interface. An alternative implementation could be to use Tensorflow/Keras or native PyTorch.

Please note that training the large BERT-model on a GPU might be a challenge. The two critical parameters are batch_size and sequence_length. Reduce these until you no longer are getting Out-of-memory(OOM) errors. The political speeches corpus above have very long sequences. You might want to truncate them at 128 tokens. This makes the task harder since the model is allowed to see less of each sequence. Reducing batch_size below 8 might lead to unstability and very long training time.

In [ ]:
from transformers.optimization import Adafactor, AdafactorSchedule

optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)

In [ ]:
# ----- Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, label = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=label, y_pred=pred)
   # recall = recall_score(y_true=labels, y_pred=pred)
   # precision = precision_score(y_true=labels, y_pred=pred)
    #accuracy = f1_score(y_true=label, y_pred=pred)

    recall = recall_score(y_true=label, y_pred=pred, average='weighted')
    precision = precision_score(y_true=label, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=label, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    #evaluation_strategy="steps",
    #eval_steps=round(steps/2),
    #logging_steps=round(steps/10),
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate, #The default here is linear decay to 0.
    warmup_steps=num_warmup_steps,
    num_train_epochs=num_epochs,
    weight_decay = weight_decay,
    #save_steps=steps, #Only saves at the end
    seed= 13, #3,
    metric_for_best_model= "accuracy",
    load_best_model_at_end=True,
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-EUJAV",
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #optimizers=(optimizer, lr_scheduler),  ##### do we want this?
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    #callbacks = [tboard_callback]
)


# Train pre-trained model
trainer.train()

In [ ]:
#import tensorflow as tf
#import datetime

# Clear any logs from previous runs
#rm -rf ./logs/

In [ ]:
#less output/checkpoint-490/trainer_state.json

In [ ]:
from google.colab import files
#files.download('output/checkpoint-392/trainer_state.json')


In [ ]:
print(train_data.shape, dev_data.shape, test_data.shape)

In [ ]:
#print(model)

In [ ]:
trainer.evaluate()

In [ ]:
#:: reinstate early stopping

# Run Preditions and print Evaluation Report
The code below first runs predictions on the train dataset. After that it prints and evaluation report using a tool from sklearn.

Typically it is two number you want from this: The accuracy score (the first number on the "accuracy"-line. In addtion most journals want you to report the F1-macro-score since this is a sequence classification task. This is the number beneath accuracy (or in the intersection between f1-score and macro-avg).

One of the tasks above is a balanced dataset. Both are binary classification. In a balanced binary classification the F1-macro and the average is basically the same (rounding differences only). In the unbalanced set, these values will vary greatly. The F1-macro is typically a much better measurement of how good your network is doing.

We repeat the same raport for eval and test as well.

In [ ]:
#Print report
from sklearn.metrics import classification_report


print("\nTest-set Evaluation")
X_test = list(test_data["text clean"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_seq_length)
test_dataset = Dataset(X_test_tokenized)
raw_pred, _, _ = trainer.predict(test_dataset)
y_pred_bool_test = np.argmax(raw_pred, axis=1)
#print(classification_report(test_data["label"], y_pred_bool, digits=3))
print(classification_report(test_data["label"].to_numpy().astype("int"), y_pred_bool_test, digits=3))

#print(classification_report(X_test["label"], y_pred_bool, digits=4))

print("\nValidation-set Evaluation")
dev_dataset = Dataset(X_val_tokenized)
w_pred, _, _ = trainer.predict(dev_dataset)
y_pred_bool_dev = np.argmax(w_pred, axis=1)
#print(classification_report(dev_data["label"], y_pred_bool, digits=3))
print(classification_report(dev_data["label"].to_numpy().astype("int"), y_pred_bool_dev, digits=3))


print("\nTrain-set Evaluation")
train_dataset = Dataset(X_train_tokenized)
train_pred, _, _ = trainer.predict(train_dataset)
y_pred_bool_train = np.argmax(train_pred, axis=1)
#print(classification_report(train_data["label"], y_pred_bool, digits=3))
print(classification_report(train_data["label"].to_numpy().astype("int"), y_pred_bool_train, digits=3))


In [ ]:
df_label_prediction = train_data.copy()

df_label_prediction['prediction'] = y_pred_bool_train.tolist()

In [ ]:
df_label_prediction.head()

In [ ]:
df_label_prediction.shape

In [ ]:
y_pred_bool_train.shape

In [ ]:
train_data.shape

In [ ]:
from google.colab import files

df_label_prediction.to_csv('trainingDataLabelsPredictions.csv')
files.download('trainingDataLabelsPredictions.csv')

In [ ]:
# Remove difficult to label data from training dataset
#indexNames = df_label_prediction[ df_label_prediction['label'] != df_label_prediction['prediction'] ].index

# Remove easy to label data from training dataset
#indexNames = df_label_prediction[ df_label_prediction['label'] == df_label_prediction['prediction'] ].index

#df_label_prediction.drop(indexNames , inplace=True)
#size=df_label_prediction.shape[0]
#orig_size = train_data.shape[0]
#print(size, (orig_size-size)/orig_size)

In [ ]:
df_label_prediction.head()

In [ ]:
# Preprocess reduced training dataset
X_train_r = list(df_label_prediction["text clean"])
y_train_r = list(df_label_prediction["label"])

X_train_r_tokenized = tokenizer(X_train_r, padding=True, truncation=True, max_length=max_seq_length)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=max_seq_length)

train_r_dataset = Dataset(X_train_r_tokenized, y_train_r)

In [ ]:
args = TrainingArguments(
    output_dir="output2",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    #evaluation_strategy="steps",
    #eval_steps=round(steps/2),
    #logging_steps=round(steps/10),
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate, #The default here is linear decay to 0.
    warmup_steps=num_warmup_steps,
    num_train_epochs=11,
    weight_decay = weight_decay,
    #save_steps=steps, #Only saves at the end
    ######seed=0,
    metric_for_best_model= "accuracy",
    load_best_model_at_end=True,
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-EUJAV",
)

In [ ]:
trainer2 = Trainer(
    model=model,
    args=args,
    train_dataset=train_r_dataset, # reduced dataset. Difficult tweets removed
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),  ##### do we want this?
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train again on reduced training dataset
trainer2.train()

In [ ]:
#files.download('output2/checkpoint-94/trainer_state.json')

In [ ]:
#less output2/checkpoint-260/trainer_state.json

In [ ]:
print("\nReduced Training-set (difficult tweets removed) Evaluation ******")
print("\nTest-set Evaluation")
raw_pred2, _, _ = trainer2.predict(test_dataset)
y_pred_bool_test2 = np.argmax(raw_pred2, axis=1)
print(classification_report(test_data["label"].to_numpy().astype("int"), y_pred_bool_test2, digits=3))

print("\nValidation-set Evaluation")
w_pred2, _, _ = trainer2.predict(dev_dataset)
y_pred_bool_dev2 = np.argmax(w_pred2, axis=1)
print(classification_report(dev_data["label"].to_numpy().astype("int"), y_pred_bool_dev2, digits=3))


print("\nTrain-set Evaluation: evaluate on reduced training dataset")
train_pred2, _, _ = trainer2.predict(train_r_dataset)
y_pred_bool_train2 = np.argmax(train_pred2, axis=1)
print(classification_report(df_label_prediction["label"].to_numpy().astype("int"), y_pred_bool_train2, digits=3))


**Confusion** matrix


In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 8
font_size = 22

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

#y_test = test_data["label"]
y_test = test_data["label"].to_numpy().astype("int")
y_pred_test = np.argmax(raw_pred2, axis=1)

cnf_matrix = confusion_matrix(y_test, y_pred_test)
cnf_matrix

In [ ]:
test_data_results = test_data.copy()
test_data_results['orig label'] = y_test
test_data_results['pred label'] = y_pred_test

In [ ]:
test_data_results.head()

In [ ]:
test_data.shape

In [ ]:
test_labels_agree_df = test_data_results[test_data_results['orig label'] == test_data_results['pred label']]
test_labels_not_agree_df = test_data_results[test_data_results['orig label'] != test_data_results['pred label']]

print(test_labels_agree_df.shape[0], test_labels_not_agree_df.shape[0])

In [ ]:
#test_labels_not_agree_df

In [ ]:
counts = np.unique(y_pred_test, return_counts=True)

In [ ]:
from sklearn.metrics import f1_score
f1_full_data = f1_score(y_test, y_pred_test, average='macro')
f1_full_data

In [ ]:
from sklearn.metrics import accuracy_score
acc_full_data = accuracy_score(y_test, y_pred_test)
acc_full_data

In [ ]:
# Test with dataset1. Trainer 2
test_data = test_data1.copy()
test_data = test_data.sample(frac=1).reset_index(drop=True)


X_test = list(test_data["text clean"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_seq_length)

test_dataset = Dataset(X_test_tokenized)
raw_pred, _, _ = trainer2.predict(test_dataset)
y_pred_test = np.argmax(raw_pred, axis=1)
y_test = test_data["label"].to_numpy().astype("int")


print(classification_report(y_test, y_pred_test, digits=3))

f1_dataset1 = f1_score(y_test, y_pred_test, average='macro')
acc_dataset1 = accuracy_score(y_test, y_pred_test)
f1_dataset1

cnf_matrix_1 = confusion_matrix(y_test, y_pred_test)
cnf_matrix_1


In [ ]:
### TEST WITH dataset 2 Trainer 2
test_data= test_data2.copy()

X_test = list(test_data["text clean"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_seq_length)

test_dataset = Dataset(X_test_tokenized)
raw_pred, _, _ = trainer2.predict(test_dataset)
y_pred_test = np.argmax(raw_pred, axis=1)
y_test = test_data["label"].to_numpy().astype("int")

print(classification_report(y_test, y_pred_test, digits=3))

f1_dataset2 = f1_score(y_test, y_pred_test, average='macro')
acc_dataset2 = accuracy_score(y_test, y_pred_test)
f1_dataset2

cnf_matrix_2 = confusion_matrix(y_test, y_pred_test)
cnf_matrix_2

In [ ]:
# Test with dataset1+2 Trainer 2
test_data= test_data1.copy()
test_data = test_data.append(test_data2)

test_data = test_data.sample(frac=1).reset_index(drop=True)


X_test = list(test_data["text clean"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_seq_length)

test_dataset = Dataset(X_test_tokenized)
raw_pred, _, _ = trainer2.predict(test_dataset)
y_pred_test = np.argmax(raw_pred, axis=1)
y_test = test_data["label"].to_numpy().astype("int")


print(classification_report(y_test, y_pred_test, digits=3))

f1_dataset12 = f1_score(y_test, y_pred_test, average='macro')
acc_dataset12 = accuracy_score(y_test, y_pred_test)
f1_dataset12

cnf_matrix_12 = confusion_matrix(y_test, y_pred_test)
cnf_matrix_12

In [ ]:
### TEST WITH SPANISH DATA

test_data = pd.read_csv(
    "https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/newconsistent_Spanish175_PromotionalNeutralDicouraging.csv",
    names=["label", "text"])

if useSpanishData:
  test_data = pd.read_csv(
      'https://raw.githubusercontent.com/cheatham1/EU-JAV-DATA/main/test_spanish_dataset.csv',
      names=["label", "text"])


test_data.label = test_data.label.astype('float').astype('Int64')
#test_data['text']  = test_data['text'].apply(lambda x: emoji.demojize(x))

print(test_data.shape)

X_test = list(test_data["text"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_seq_length)

test_dataset = Dataset(X_test_tokenized)
raw_pred, _, _ = trainer2.predict(test_dataset)

y_pred_test = np.argmax(raw_pred, axis=1)
y_test = test_data["label"].to_numpy().astype("int")

#print(classification_report(test_data["label"].to_numpy().astype("int"), y_pred_test, digits=3))
print(classification_report(y_test, y_pred_test, digits=3))

f1_spanish_data = f1_score(y_test, y_pred_test, average='macro')
acc_spanish_data = accuracy_score(y_test, y_pred_test)

In [ ]:
print("Results from TEST dataset")
print("Accuracy: ")
print("Full test data: {:.3f}".format(acc_dataset12))
print("Dataset1: {:.3f}".format( acc_dataset1))
print("Dataset2: {:.3f}".format( acc_dataset2))
print("Spanish data: {:.3f}".format( acc_spanish_data))


print("F1")

print("Full test data: {:.3f}".format(f1_dataset12))
print("Dataset1: {:.3f}".format( f1_dataset1))
print("Dataset2: {:.3f}".format( f1_dataset2))
print("Spanish data: {:.3f}".format( f1_spanish_data))


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")

    print(cm)
    print("\n")

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar(shrink=0.7)
    tick_marks = np.arange(len(classes))
    #plt.xticks(tick_marks, classes, rotation=45)
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)


    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=font_size)

    plt.tight_layout()
    plt.ylabel('True label',fontsize=font_size)
    plt.xlabel('Predicted label',fontsize=font_size)

In [ ]:
import itertools
#plt.figure()
#class_names= ["Promotional","Neutral","Discouraging","Ambiguous","Indiscernable"]

class_names= ["Promotional","Neutral","Discouraging"]

plot_confusion_matrix(cnf_matrix, classes= class_names, normalize=True)
plt.title('Normalized Confusion Matrix',fontsize=font_size)


In [ ]:
cnf_matrix

In [ ]:
plot_confusion_matrix(cnf_matrix_1, classes= class_names, normalize=True)
plt.title('Normalized Confusion Matrix: Test dataset1',fontsize=font_size)


In [ ]:
plot_confusion_matrix(cnf_matrix_2, classes= class_names, normalize=True)
plt.title('Normalized Confusion Matrix: Test dataset2',fontsize=font_size)


In [ ]:
plot_confusion_matrix(cnf_matrix_12, classes= class_names, normalize=True)
plt.title('Normalized Confusion Matrix: Test dataset1+2',fontsize=font_size)


In [ ]:
# Tokemizer
#tokenizer.tokenize("#no-vax #novax")

In [ ]:
#X_train_tokenized

In [ ]:
#!ls output/checkpoint-1/

# **Save the model**

In [ ]:
::

In [ ]:
### THIS NEEDS TO BE RUN to save model
### Install git lfs
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

In [ ]:
# save on huggingface

In [ ]:
!git config --global user.email "Susancheatham1@gmail.com"

In [ ]:
!huggingface-cli login

In [ ]:
!git config --global credential.helper store

In [ ]:
model_name_to_save = "xlm-roberta-large-finetuned-d1r01"

In [ ]:
# Save locally first
model.save_pretrained('./xlm-roberta-large-finetuned-d1r01/')

In [ ]:
tokenizer.save_pretrained('./xlm-roberta-large-finetuned-d1r01/')

In [ ]:
trainer.save_model('./xlm-roberta-large-finetuned-d1r01/')

In [ ]:
!ls xlm-roberta-large-finetuned-r993/

In [ ]:
    # Files to expect...
    # a config.json file, which saves the configuration of your model ;
    # a pytorch_model.bin file, which is the PyTorch checkpoint (unless you can’t have it for some reason) ;
    # a tf_model.h5 file, which is the TensorFlow checkpoint (unless you can’t have it for some reason) ;
    # a special_tokens_map.json, which is part of your tokenizer save;
    # a tokenizer_config.json, which is part of your tokenizer save;
    # files named vocab.json, vocab.txt, merges.txt, or similar, which contain the vocabulary of your tokenizer, part of your tokenizer save;
    # maybe a added_tokens.json, which is part of your tokenizer save.


In [ ]:
!huggingface-cli repo create "xlm-roberta-large-finetuned-d1r01" --yes


In [ ]:
model.push_to_hub('https://huggingface.co/Cheatham/xlm-roberta-large-finetuned-d1r01/')
tokenizer.push_to_hub('https://huggingface.co/Cheatham/xlm-roberta-large-finetuned-d1r01/')


In [ ]:
#####import os
#### Mount Google Drive to this Notebook instance.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls drive/MyDrive/EU-JAV/Models

In [ ]:
cd MyDrive/EU-JAV/Models

In [ ]:
trainer.save_model("EU-JAV-finetuned-xlmroberta")

In [ ]:
#tokenizer.save_pretrained("EU-JAV-finetuned-xlmroberta-tokenizer")


In [ ]:
#model.save_pretrained("EU-JAV-finetuned-xlmroberta-model")

In [ ]:
###drive.flush_and_unmount()

In [ ]:
#ls EU-JAV-models/EUJAV-tokenizer

In [ ]:
#ls EU-JAV-models/EUJAV-finetuned-roberta-model_uncased

In [ ]:
#!git lfs install

In [ ]:
!git config --global credential.helper store

In [ ]:

model.save_pretrained('https://huggingface.co/Cheatham/xlm-roberta-large-finetuned4/')
trainer.save_model('https://huggingface.co/Cheatham/xlm-roberta-large-finetuned4/')
tokenizer.save_pretrained('https://huggingface.co/Cheatham/xlm-roberta-large-finetuned4/')


In [ ]:
!git config --global user.email susancheatham1@gmail.com

In [ ]:
model.save_pretrained('https://github.com/cheatham1/EU-JAV-Models/xlm-roberta-large-finetuned4')
trainer.save_model("https://github.com/cheatham1/EU-JAV-Models/xlm-roberta-large-finetuned4")
tokenizer.save_pretrained("https://github.com/cheatham1/EU-JAV-Models/xlm-roberta-large-finetuned4")

In [ ]:
!git init

In [ ]:
!git status

In [ ]:
!git add xlm-roberta-large-finetuned4

In [ ]:
#
!git status


In [ ]:
!git commit -m "EU-JAV model update"
!git push

In [ ]:
!git push --set-upstream origin main

In [ ]:
!git remote add origin https://github.com/cheatham1/EU-JAV-Models.git
!git branch -M main
!git push -u origin main

In [ ]:
!git remote set-url origin git@github.com:cheatham1/EU-JAV-Models.git